In [1]:
import pandas as pd
import numpy as np
import copy
import warnings
import plotly
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from sklearn import metrics
from sklearn.model_selection import train_test_split
from PIL import Image 
import random
warnings.filterwarnings("ignore")
from tqdm import tqdm
import string

import torch
from torch import nn
from transformers import AutoTokenizer
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModel, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

# Data

In [5]:
news = pd.read_csv('../Arts.csv', sep=';')

In [6]:
news.columns

Index(['Web', 'Title', 'Url', 'Datetime', 'Abstract'], dtype='object')

In [7]:
news.head()

,Web,Title,Url,Datetime,Abstract
0,idnes.cz,Česko vstoupilo do roku 2023. Zákaz pyrotechniky v centru Prahy nezabral,https://www.idnes.cz/praha/zpravy/silvestr-novy-rok-hasici-policie-zachranna-sluzba-pripravy-vyjezdy-pyrotechnika.A221231_145404_domaci_pitt,2023-01-01,"Česká republika vstoupila do nového roku 2023. Oslavy se konaly na řadě míst, tradičně největší byly v centru Prahy, kde i přes zákaz došlo k odpalování pyrotechniky. Na Václavském náměstí před půlnocí muž kopl do hlavy policistu. Do druhé hodiny ranní zasahovali hasiči u padesátky požárů, záchranáři využili i specializované vozidlo Golem."
1,idnes.cz,PŘEHLEDNĚ: Co nového nás čeká v roce 2023. Za co více zaplatíme i dostaneme,https://www.idnes.cz/zpravy/domaci/cesko-zmeny-v-roce-2023-prehledne.A221230_105559_domaci_knn,2023-01-01,"Česko si zvolí nového prezidenta, na pražském magistrátu usedne nový primátor. Řada lidí bude muset za pohotovostí cestovat do jiného města, statisíce motoristů zase čeká výměna řidičských průkazů. To vše přinese rok 2023. Redakce iDNES.cz přináší přehledně seznam hlavních změn."
2,idnes.cz,"Postavme se dezinformátorům a populistům, vyzval šéf Senátu Vystrčil",https://www.idnes.cz/zpravy/domaci/novorocni-projev-2023-predseda-senatu-vystrcil-premier-fiala.A221231_161639_domaci_kop,2023-01-01,"S novoročním projevem vystoupil předseda Senátu Miloš Vystrčil z ODS. „Česká republiko, dobrý den,“ zahájil projev stejnými slovy jako před třiceti lety tehdejší šéf Senátu Milan Uhde v den, kdy po rozdělení Československa vznikla samostatná Česká republika. Vystrčil ocenil poklidný způsob rozdělení společného státu Čechů a Slováků před 30 lety."
3,idnes.cz,Požárů na silvestra bylo o třetinu víc než loni. Hasiči zachraňovali i z komína,https://www.idnes.cz/zpravy/domaci/silvestr-zachranna-sluzba-zachranka-ohnostroj-petardy-zraneni-zasah-noc-novy-rok.A230101_073804_domaci_klf,2023-01-01,Hasiči v Česku během silvestrovské noci zasahovali u 286 požárů. Je to asi o třetinu víc než loni. Zásahy u požárů také tvořily tři čtvrtiny všech výjezdů hasičů. Záchranáři v krajích pak kromě zranění petardami ošetřovali i oči poraněné při otevírání sektu a opařené dítě.
4,idnes.cz,"Hodnocení skončeného předsednictví: Češi uspěli, ale neumí se tím pochlubit",https://www.idnes.cz/zpravy/domaci/ceske-predsednictvi-v-rade-eu-hodnoceni.A221213_104017_domaci_knn,2023-01-01,"Spolu s koncem roku předala Česká republika pomyslné žezlo předsednictví v Radě EU Švédsku. Tím zakončila půl rok, během kterého zodpovídala za její fungování a stála tak „v čele Evropy“. Podle odborníků i politiků, které redakce iDNES.cz oslovila, země v této zkoušce obstála. Úspěchy jsou ale málo vidět a vláda je nedokázala dostatečně „prodat“."


# Preproc

In [8]:
news[news["Abstract"].isna()]

,Web,Title,Url,Datetime,Abstract
1437,lidovky.cz,Hlásíte se z páté třídy na víceleté gymnázium,https://www.lidovky.cz/serialy/gymnazium-patak-prijimaci-zkousky-test-lekce-predmet.A230112_110310_ln_specialy_ape,2023-01-20,NaN
1438,lidovky.cz,Hlásíte se z deváté třídy na střední školu,https://www.lidovky.cz/serialy/stredni-skola-gymnazium-prijimaci-zkousky-test-nanecisto-lekce.A230112_110519_ln_specialy_ape,2023-01-20,NaN
2022,aktualne.cz,"Zeman udělil medaili dobrovolnému hasiči, který zahynul při požáru",https://zpravy.aktualne.cz//domaci/zeman-udelil-medaili-dobrovolnemu-hasici-ktery-zahynul-pri-p/r~6923cfc889b211ed93abac1f6b220ee8/,2023-01-01,NaN
2023,aktualne.cz,"V Česku i na Nový rok padají teplotní rekordy, místy už bylo přes 16 stupňů",https://zpravy.aktualne.cz//domaci/v-cesku-i-na-novy-rok-padaji-teplotni-rekordy-misty-uz-bylo/r~0f4aed8089b511eda3c0ac1f6b220ee8/,2023-01-01,NaN
2024,aktualne.cz,"Silvestrovské oslavy záchranné složky zvládly, nejkurióznější byla srážka auta s pštrosem",https://zpravy.aktualne.cz//domaci/silvestrovske-oslavy-zachranne-slozky-zvladly-nejkurioznejsi/r~e0f0e0a489c611edba5b0cc47ab5f122/,2023-01-01,NaN
2034,aktualne.cz,Policisté v kauze Dozimetr získají spolupracujícího obviněného,https://zpravy.aktualne.cz//domaci/policiste-v-kauze-dozimetr-ziskaji-spolupracujiciho-obvinene/r~a79e64268ac411edbe29ac1f6b220ee8/,2023-01-02,NaN
2035,aktualne.cz,Zeman se ve středu sejde s kandidátem na ministra životního prostředí Petrem Hladíkem,https://zpravy.aktualne.cz//domaci/zeman-se-ve-stredu-sejde-s-kandidatem-na-ministra-zivotniho/r~aa423df48abc11eda3c0ac1f6b220ee8/,2023-01-02,NaN
2036,aktualne.cz,"Teplotní rekordy padaly i v pondělí, na Jesenicku meteorologové naměřili přes 18,5 stupně",https://zpravy.aktualne.cz//domaci/teplotni-rekordy-padaly-i-v-pondeli-na-jesenicku-meteorologo/r~b72f29d28b2f11ed8d680cc47ab5f122/,2023-01-03,NaN
2037,aktualne.cz,"STAN zvažují výměnu ministra Balaše, chtějí ho poslat na Ústavní soud",https://zpravy.aktualne.cz//domaci/stan-zvazuji-vymenu-ministra-balase-chteji-ho-poslat-na-usta/r~304627268b3a11ed8980ac1f6b220ee8/,2023-01-03,NaN
2038,aktualne.cz,Na pohřbu bývalého papeže Benedikta XVI. zastoupí Česko Fiala,https://zpravy.aktualne.cz//domaci/na-pohrbu-byvaleho-papeze-benedikta-xvi-zastoupi-cesko-fiala/r~64b61f628b5211ed8b4e0cc47ab5f122/,2023-01-03,NaN


In [9]:
print('the number of articles before processing :',len(news))
news = news.dropna(subset=['Abstract'])
print('The number of articles after processing :',len(news))

the number of articles before processing : 6126
The number of articles after processing : 5962


In [10]:
duplicate_rows = news[news.duplicated()]
print("Duplicate Rows:")
print(duplicate_rows)

Duplicate Rows:
                    Web                                             Title  \
2731  czechfreepress.cz  Za našimi zády se dějí věci, je to smutné čtení…   

                                                                                              Url  \
2731  https://www.czechfreepress.cz//politika/za-nasimi-zady-se-deji-veci-je-to-smutne-cteni.html   

        Datetime  \
2731  2023-01-15   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               Abstract  
2731  Zatímco veřejnost se zabývá prezidentskými volbami, Pětikoalice v jejich stínu usilovně p

In [11]:
print('The number of articles before processing :',len(news))
news = news.drop_duplicates()
print('The number of articles after processing :',len(news))

The number of articles before processing : 5962
The number of articles after processing : 5961


In [12]:
news.reset_index(drop=True, inplace=True)

In [13]:
news.head()

,Web,Title,Url,Datetime,Abstract
0,idnes.cz,Česko vstoupilo do roku 2023. Zákaz pyrotechniky v centru Prahy nezabral,https://www.idnes.cz/praha/zpravy/silvestr-novy-rok-hasici-policie-zachranna-sluzba-pripravy-vyjezdy-pyrotechnika.A221231_145404_domaci_pitt,2023-01-01,"Česká republika vstoupila do nového roku 2023. Oslavy se konaly na řadě míst, tradičně největší byly v centru Prahy, kde i přes zákaz došlo k odpalování pyrotechniky. Na Václavském náměstí před půlnocí muž kopl do hlavy policistu. Do druhé hodiny ranní zasahovali hasiči u padesátky požárů, záchranáři využili i specializované vozidlo Golem."
1,idnes.cz,PŘEHLEDNĚ: Co nového nás čeká v roce 2023. Za co více zaplatíme i dostaneme,https://www.idnes.cz/zpravy/domaci/cesko-zmeny-v-roce-2023-prehledne.A221230_105559_domaci_knn,2023-01-01,"Česko si zvolí nového prezidenta, na pražském magistrátu usedne nový primátor. Řada lidí bude muset za pohotovostí cestovat do jiného města, statisíce motoristů zase čeká výměna řidičských průkazů. To vše přinese rok 2023. Redakce iDNES.cz přináší přehledně seznam hlavních změn."
2,idnes.cz,"Postavme se dezinformátorům a populistům, vyzval šéf Senátu Vystrčil",https://www.idnes.cz/zpravy/domaci/novorocni-projev-2023-predseda-senatu-vystrcil-premier-fiala.A221231_161639_domaci_kop,2023-01-01,"S novoročním projevem vystoupil předseda Senátu Miloš Vystrčil z ODS. „Česká republiko, dobrý den,“ zahájil projev stejnými slovy jako před třiceti lety tehdejší šéf Senátu Milan Uhde v den, kdy po rozdělení Československa vznikla samostatná Česká republika. Vystrčil ocenil poklidný způsob rozdělení společného státu Čechů a Slováků před 30 lety."
3,idnes.cz,Požárů na silvestra bylo o třetinu víc než loni. Hasiči zachraňovali i z komína,https://www.idnes.cz/zpravy/domaci/silvestr-zachranna-sluzba-zachranka-ohnostroj-petardy-zraneni-zasah-noc-novy-rok.A230101_073804_domaci_klf,2023-01-01,Hasiči v Česku během silvestrovské noci zasahovali u 286 požárů. Je to asi o třetinu víc než loni. Zásahy u požárů také tvořily tři čtvrtiny všech výjezdů hasičů. Záchranáři v krajích pak kromě zranění petardami ošetřovali i oči poraněné při otevírání sektu a opařené dítě.
4,idnes.cz,"Hodnocení skončeného předsednictví: Češi uspěli, ale neumí se tím pochlubit",https://www.idnes.cz/zpravy/domaci/ceske-predsednictvi-v-rade-eu-hodnoceni.A221213_104017_domaci_knn,2023-01-01,"Spolu s koncem roku předala Česká republika pomyslné žezlo předsednictví v Radě EU Švédsku. Tím zakončila půl rok, během kterého zodpovídala za její fungování a stála tak „v čele Evropy“. Podle odborníků i politiků, které redakce iDNES.cz oslovila, země v této zkoušce obstála. Úspěchy jsou ale málo vidět a vláda je nedokázala dostatečně „prodat“."


# Tokenizace

In [14]:
bert_uncased_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [15]:
class BertBaseDataset(Dataset):
    def __init__(self, tokenizer, df, text_col, max_len):
        self.tokenizer = tokenizer
        self.texts = df[text_col].values.tolist()
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        res = self.tokenizer(
            self.texts[idx],
            return_tensors="pt",
            padding=False,
            truncation=True,
            # return_special_tokens_mask=True,
            max_length=self.max_len,
        )
        res = {k:v[0] for k,v in res.items()}
        return res
    
def dynamic_pad_data_loader(tokenizer, dataset, batch_size, shuffle):
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    
    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        collate_fn=data_collator
    )

In [16]:
backbone = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(backbone)

ds = BertBaseDataset(tokenizer, news, "Abstract", None)
dl = dynamic_pad_data_loader(tokenizer, ds, batch_size=32, shuffle=False)

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

# Sentiment Analyza

In [17]:
model = AutoModelForSequenceClassification.from_pretrained(backbone)
model.eval()

preds = []

with torch.no_grad(), tqdm(total=len(dl)) as pbar:
    for batch in dl:
        output = model(**batch)
        preds.append(output["logits"])
        pbar.update(1)

# Concatenate the logits from all batches
preds = torch.cat(preds, dim=0)

print("Shape of all logits:", preds.shape)

100%|█████████████████████████████████████████| 187/187 [39:53<00:00, 12.80s/it]

Shape of all logits: torch.Size([5961, 5])


# Vysledky

In [18]:
preds = preds.softmax(1).numpy()
preds_df = pd.DataFrame(preds, columns=[f"{i+1} stars" for i in range(5)])
preds_df["predicted stars"] = preds.argmax(axis=1) + 1
preds_df.insert(0, "Title", news['Title'])
preds_df.insert(0, "Abstract", news["Abstract"])
preds_df.insert(0, "Datetime", news["Datetime"])
preds_df.insert(0, 'Web', news['Web'])
preds_df.head()

,Web,Datetime,Abstract,Title,1 stars,2 stars,3 stars,4 stars,5 stars,predicted stars
0,idnes.cz,2023-01-01,"Česká republika vstoupila do nového roku 2023. Oslavy se konaly na řadě míst, tradičně největší byly v centru Prahy, kde i přes zákaz došlo k odpalování pyrotechniky. Na Václavském náměstí před půlnocí muž kopl do hlavy policistu. Do druhé hodiny ranní zasahovali hasiči u padesátky požárů, záchranáři využili i specializované vozidlo Golem.",Česko vstoupilo do roku 2023. Zákaz pyrotechniky v centru Prahy nezabral,0.213713,0.120329,0.124306,0.232894,0.308758,5
1,idnes.cz,2023-01-01,"Česko si zvolí nového prezidenta, na pražském magistrátu usedne nový primátor. Řada lidí bude muset za pohotovostí cestovat do jiného města, statisíce motoristů zase čeká výměna řidičských průkazů. To vše přinese rok 2023. Redakce iDNES.cz přináší přehledně seznam hlavních změn.",PŘEHLEDNĚ: Co nového nás čeká v roce 2023. Za co více zaplatíme i dostaneme,0.088154,0.072447,0.159511,0.369156,0.310732,4
2,idnes.cz,2023-01-01,"S novoročním projevem vystoupil předseda Senátu Miloš Vystrčil z ODS. „Česká republiko, dobrý den,“ zahájil projev stejnými slovy jako před třiceti lety tehdejší šéf Senátu Milan Uhde v den, kdy po rozdělení Československa vznikla samostatná Česká republika. Vystrčil ocenil poklidný způsob rozdělení společného státu Čechů a Slováků před 30 lety.","Postavme se dezinformátorům a populistům, vyzval šéf Senátu Vystrčil",0.183144,0.164758,0.154964,0.238330,0.258804,5
3,idnes.cz,2023-01-01,Hasiči v Česku během silvestrovské noci zasahovali u 286 požárů. Je to asi o třetinu víc než loni. Zásahy u požárů také tvořily tři čtvrtiny všech výjezdů hasičů. Záchranáři v krajích pak kromě zranění petardami ošetřovali i oči poraněné při otevírání sektu a opařené dítě.,Požárů na silvestra bylo o třetinu víc než loni. Hasiči zachraňovali i z komína,0.785202,0.121507,0.043305,0.019971,0.030014,1
4,idnes.cz,2023-01-01,"Spolu s koncem roku předala Česká republika pomyslné žezlo předsednictví v Radě EU Švédsku. Tím zakončila půl rok, během kterého zodpovídala za její fungování a stála tak „v čele Evropy“. Podle odborníků i politiků, které redakce iDNES.cz oslovila, země v této zkoušce obstála. Úspěchy jsou ale málo vidět a vláda je nedokázala dostatečně „prodat“.","Hodnocení skončeného předsednictví: Češi uspěli, ale neumí se tím pochlubit",0.317008,0.359434,0.207930,0.082485,0.033143,2


In [19]:
len(preds_df)

5961

In [24]:
preds_df.tail(100)

,Web,Datetime,Abstract,Title,1 stars,2 stars,3 stars,4 stars,5 stars,predicted stars
5861,parlamentnilisty.cz,2023-01-28,"Známý hudebník, skladatel a člen legendárního Pražského výběru Michael Kocáb – který na sebe upoutal pozornost prohlášením, že nebude volit ani Andreje Babiše, ani Petra Pavla, protože se jedná o dva komunisty – se opět vyjádřil k prezidentské volbě. Tentokrát Kocáb pro CNN Prima News sdělil, že je smutné, že se národ rozdělil kvůli dvěma komunistům. A mj. zaznělo, že atmosféru strachu využíval i volební tým Petra Pavla.","Národ se hašteří kvůli dvěma komunistům? Drsné, žasne Kocáb",0.120335,0.163790,0.216037,0.290295,0.209543,4
5862,parlamentnilisty.cz,2023-01-28,"Prezidentské volby vrcholí. Příznivci obou kandidátů dávají znát, na čí straně jsou jejich sympatie. Česká televize si pro svoje natáčení vybrala dva zástupce herecké profese. Eva Holubová byla často k vidění i na veřejnosti jako velká podporovatelka Petra Pavla, zatímco Ivan Vyskočil se netajil svými sympatiemi k Andreji Babišovi. V případě neúspěchu svého kandidáta je Holubová připravena dál bojovat. Vyskočil bude vítěze volby s úctou respektovat.",Vyskočila pustili do ČT. Nabádal k úctě. Holubová za „tým generál“ hrozila pokračováním války,0.048403,0.120491,0.249413,0.437408,0.144285,4
5863,parlamentnilisty.cz,2023-01-28,"TÝDEN V MÉDIÍCH Pokrytectví rádoby chlapáků v politice, ať už jde o předsedu vlády nebo o kandidáta na Pražský hrad se šmouhou kariérního vojáka minulého režimu, který byl zákonem označen za protiprávní, trefně vystihl komentář, jejž Petr Žantovský označil za nejlepší mediální počin uplynulého týdne. Zmínění hrdinové sice výhrůžky násilím či smrtí odsoudí, ale jedním dechem jakoby dodají, že si za to může oběť sama a že ty „dobré lidi“ hnusně a nedemokraticky vyprovokovala.",Může si za to sám... „Humánní“ tábor se tento týden předvedl v plné kráse,0.343541,0.231932,0.167783,0.116739,0.140005,1
5864,parlamentnilisty.cz,2023-01-28,"První den závěrečného kola prezidentských voleb v Česku ve 22:00 skončil. Zájem voličů byl často vyšší než před dvěma týdny v úvodním kole hlasování o tom, kdo bude moci vystřídat Miloše Zemana v čele státu. Lidé, kteří dnes hlasovat nestihli, budou mít možnost vybrat mezi Petrem Pavlem a Andrejem Babišem ještě v sobotu od 08:00 do 14:00.",O druhé kolo je mimořádný zájem. Polovina ze všech voličů už má odvoleno,0.363430,0.375709,0.169139,0.061443,0.030279,2
5865,parlamentnilisty.cz,2023-01-28,"„Zase by se objevila hesla typu ‚Vše pro frontu‘, nemluvě o tom, že pro ruské zbraně by Česko nepředstavovalo vzdálený týl, ale hlubší frontu.“ Politolog profesor Oskar Krejčí analyzuje slova náčelníka generálního štábu o případné výběrové mobilizaci. „Nikdy v dějinách jsme neobstáli. Česko je země, která má především budovat integrovaný záchranný systém,“ připomíná. Navíc Česko půjde do války, když tak rozhodne NATO – tedy Washington. „Vojenské aliance mají velitele,“ říká Krejčí.","Nezodpovědní váleční nadšenci z Prahy... Profesor Krejčí vysvětluje, co obnášejí Řehkovy plány",0.274266,0.318154,0.214224,0.128074,0.065282,2
5866,parlamentnilisty.cz,2023-01-27,"„Co se stane, pokud vyhraje Andrej Babiš?“ Přesně takovouto otázku, zamýšlející se nad případnou výhrou expremiéra a předsedy hnutí ANO v druhém kole prezidentských voleb, si aktuálně pokládá množství politiků i komentujících na sociálních sítích. Padají tak názory, že by Milion chvilek „řetězem blokoval Hrad“, či to, že pak „nabere antikomunismus na obrátkách“. Řeporyjský starosta Pavel Novotný (ODS) má však jasno. Zítra podle něj bude „pro lízače miliardářské zadele pěknej kravál“. „Kuniksichta“ Babiše podle něj čeká u voleb „výprask“.","Už se vymýšlí, co když vyhraje Babiš: Fiala svolává v noci vládu, řetězová blokáda Hradu...",0.357804,0.310450,0.178141,0.106949,0.046656,1
5867,parlamentnilisty.cz,2023-01-27,"První den závěrečného kola prezidentských voleb v Česku ve 22.00 skončil. Zájem voličů byl často vyšší než před dvěma týdny v úvodním kole hlaso

In [23]:
csv_path = "../data/preds_abstract_web_bert_new.csv"

# Save the DataFrame to a CSV file
preds_df.to_csv(csv_path, index=False)